In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import pandas as pd
import os

In [ ]:
# Example usage: Specify dataset path and data range [start_idx:end_idx]
dataset_path = 'power.csv'
data_start_idx = 0
data_end_idx = 10000

In [ ]:
print(torch.cuda.is_available())

In [ ]:
file_path = '/path/to/your/power_train.pth'

In [ ]:


def train_and_save_model(dataset_path, start_idx, end_idx, device):
    # Load your dataset
    df = pd.read_csv(dataset_path)

    # Select the specified range of data
    selected_data = df.iloc[start_idx:end_idx]

    # Initialize BERT tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Tokenize input texts and prepare input tensors
    tokenized_texts = selected_data['text_en'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=256))
    max_len = max(len(tokens) for tokens in tokenized_texts)
    padded_sequences = [tokens + [0]*(max_len - len(tokens)) for tokens in tokenized_texts]
    input_ids = torch.tensor(padded_sequences).to(device)  # Move input to device
    labels = torch.tensor(selected_data['label'].values).to(device)  # Move labels to device

    # Create TensorDataset and DataLoader for training
    dataset = TensorDataset(input_ids, labels)
    batch_size = 16
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Load pre-trained BERT model for sequence classification
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)  # Move model to device


    if os.path.exists(file_path) and file_path.lower().endswith('.pth'):

        # Load pre-trained model state if available
        ###############################################################################
        model.load_state_dict(torch.load('power_train.pth', map_location=device))
        ###############################################################################

    # Define optimizer and learning rate
    optimizer = AdamW(model.parameters(), lr=2e-5)

    # Training loop
    num_epochs = 16
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            batch_input_ids, batch_labels = batch
            batch_input_ids, batch_labels = batch_input_ids.to(device), batch_labels.to(device)  # Move batch to device
            outputs = model(batch_input_ids, labels=batch_labels)
            loss = outputs.loss

            total_loss += loss.item()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}: Average Loss = {total_loss / len(train_loader)}")

    # Save the final trained model state dictionary
    final_model_state_dict_path = 'power_train.pth'
    torch.save(model.state_dict(), final_model_state_dict_path)

    return final_model_state_dict_path

# Example usage:
cuda_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

trained_model_path = train_and_save_model(dataset_path, data_start_idx, data_end_idx, cuda_device)
print(f"Trained model saved at: {trained_model_path}")
